In [3]:
%matplotlib inline
import pickle
import os.path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle, Arc

In [4]:
def load_data(dirname, filename):
    currdir = os.getcwd()
    datadir = os.path.abspath(os.path.join(currdir, dirname))
    return os.path.abspath(os.path.join(datadir, filename))

def generate_xy_grids(xrange, yrange):
    x = np.arange(xrange[0], xrange[1], xrange[2])
    y = np.arange(yrange[0], yrange[1], yrange[2])

    return np.meshgrid(x, y)

In [3]:
def draw_court(ax=None, color='black', lw=2, outer_lines=False):
    # If an axes object isn't provided to plot onto, just get current one
    if ax is None:
        ax = plt.gca()

    # Create the various parts of an NBA basketball court

    # Create the basketball hoop
    # Diameter of a hoop is 46cm so it has a radius of 23cm, which is a value
    # 2.3 in our coordinate system
    hoop = Circle((0, 0), radius=2.3, linewidth=lw, color=color, fill=False)

    # Create backboard
    backboard = Rectangle((-9, -2.3), 18, -0.1, linewidth=lw, color=color)

    # The paint
    # Create the outer box 0f the paint, width=4.9m, height=5.8m
    outer_box = Rectangle((-24.45, -14.3), 49, 58, linewidth=lw, color=color,
                          fill=False)
    # Create the inner box of the paint, widt=3.6ft, height=5.8ft
    inner_box = Rectangle((-18, -14.3), 36, 58, linewidth=lw, color=color,
                          fill=False)

    # Create free throw top arc
    top_free_throw = Arc((0, 43.7), 36, 36, theta1=0, theta2=180,
                         linewidth=lw, color=color, fill=False)
    # Create free throw bottom arc
    bottom_free_throw = Arc((0, 43.7), 36, 36, theta1=180, theta2=0,
                            linewidth=lw, color=color, linestyle='dashed')
    # Restricted Zone, it is an arc with 4ft radius from center of the hoop
    restricted = Arc((0, 0), 25, 25, theta1=0, theta2=180, linewidth=lw,
                     color=color)

    # Three point line
    # Create the side 3pt lines, they are 2.68m long before they begin to arc
    corner_three_a = Rectangle((-66, -14.3), 0, 28.8, linewidth=lw,
                               color=color)
    corner_three_b = Rectangle((66, -14.3), 0, 28.8, linewidth=lw, color=color)
    # 3pt arc - center of arc will be the hoop, arc is 6.75m away from hoop
    # I just played around with the theta values until they lined up with the 
    # threes
    three_arc = Arc((0, 0), 135, 135, theta1=12, theta2=168, linewidth=lw,
                    color=color)

    # Center Court
    center_outer_arc = Arc((0, 125.7), 36, 36, theta1=180, theta2=0,
                           linewidth=lw, color=color)
    center_inner_arc = Arc((0, 125.7), 12, 12, theta1=180, theta2=0,
                           linewidth=lw, color=color)

    # List of the court elements to be plotted onto the axes
    court_elements = [
        hoop,
        backboard,
        outer_box,
        inner_box,
        top_free_throw,
        bottom_free_throw,
        restricted,
        corner_three_a,
        corner_three_b,
        three_arc,
        center_outer_arc,
        center_inner_arc
    ]

    if outer_lines:
        # Draw the half court line, baseline and side out bound lines
        outer_lines = Rectangle((-75, -14.3), 150, 140, linewidth=lw,
                                color=color, fill=False)
        court_elements.append(outer_lines)

    # Add the court elements onto the axes
    for element in court_elements:
        ax.add_patch(element)

    return ax

In [5]:
dirname = '../../../outputs/'
ebppa = 'ebppa/ebppa.csv'
outdir = '../../../outputs/local-stats'

playersfg = 'players-fg/players-fg-retained.csv'

df_fg = pd.read_csv(load_data(dirname, playersfg))
players = df_fg['player'].unique()
xx,yy = generate_xy_grids((-75,75,5), (-15,125,5))

filehandler = open('{}/players-local-stats.obj'.format(outdir), 'rb') 
df_stats_players = pickle.load(filehandler)

In [22]:
grid_y, grid_x = xx.shape

for p in players:
    df_x = df_stats_players[df_stats_players.player==p]
    team = df_x.team.values[0]

    loc = []

    df_x_ssce = df_x.grid_ssce
    df_x_prla = df_x.grid_prla
    df_x_fga = df_x.grid_fga
    df_x_fg = df_x.grid_fg
    df_x_pts = df_x.grid_pts
    # df_x_ppa = df_x.grid_ppa

    x_ssce = df_x_ssce.to_numpy()[0]
    x_prla = df_x_prla.to_numpy()[0]
    x_fga = df_x_fga.to_numpy()[0]
    x_fg = df_x_fg.to_numpy()[0]
    x_pts = df_x_pts.to_numpy()[0]
    # x_ppa = df_x_ppa.to_numpy()[0]

    for y in range(grid_y):
        for x in range(grid_x):
            locy = yy[y][x]
            locx = xx[y][x]
            dist = round(np.sqrt((locy**2)+(locx**2))/10,2)
            fga = x_fga[y][x]
            fg = x_fg[y][x]
            pts = x_pts[y][x]
            ssce = x_ssce[y][x]
            prla = x_prla[y][x]
    #         ppa = x_ppa[y][x]
            loc.append([locy + 2.5, locx + 2.5, fga, fg, pts, ssce, prla, dist])


    pd.DataFrame(loc, columns=['y', 'x', 'fga', 'fg', 'pts', 'ssce', 'prla', 'dist']).to_csv('{}/to-map/players/{}-to-map.csv'.format(outdir, p), index=False)

In [7]:
filehandler_teams = open('{}/teams-local-stats.obj'.format(outdir), 'rb') 
df_stats_teams = pickle.load(filehandler_teams)

teams = df_stats_teams.team.unique()

tms = []

grid_y, grid_x = xx.shape

for p in teams:
    df_x = df_stats_teams[df_stats_teams.team==p]
    team = p

    loc = []

    df_x_ssce = df_x.grid_ssce
    df_x_prla = df_x.grid_prla
    df_x_fga = df_x.grid_fga
    df_x_fg = df_x.grid_fg
    df_x_pts = df_x.grid_pts
    # df_x_ppa = df_x.grid_ppa

    x_ssce = df_x_ssce.to_numpy()[0]
    x_prla = df_x_prla.to_numpy()[0]
    x_fga = df_x_fga.to_numpy()[0]
    x_fg = df_x_fg.to_numpy()[0]
    x_pts = df_x_pts.to_numpy()[0]
    # x_ppa = df_x_ppa.to_numpy()[0]

    for y in range(grid_y):
        for x in range(grid_x):
            locy = yy[y][x]
            locx = xx[y][x]
            dist = round(np.sqrt((locy**2)+(locx**2))/10,2)
            fga = x_fga[y][x]
            fg = x_fg[y][x]
            pts = x_pts[y][x]
            ssce = x_ssce[y][x]
            prla = x_prla[y][x]
    #         ppa = x_ppa[y][x]
            loc.append([locy + 2.5, locx + 2.5, fga, fg, pts, ssce, prla, dist])


    pd.DataFrame(loc, columns=['y', 'x', 'fga', 'fg', 'pts', 'ssce', 'prla', 'dist']).to_csv('{}/to-map/teams/{}-to-map.csv'.format(outdir, p), index=False)

In [8]:
filehandler_teams_opp = open('{}/teams-opp-local-stats.obj'.format(outdir), 'rb') 
df_stats_teams_opp = pickle.load(filehandler_teams_opp)

teams = df_stats_teams_opp.team.unique()

tms = []

grid_y, grid_x = xx.shape

for p in teams:
    df_x = df_stats_teams_opp[df_stats_teams_opp.team==p]
    team = p

    loc = []

    df_x_ssce = df_x.grid_ssce
    df_x_prla = df_x.grid_prla
    df_x_fga = df_x.grid_fga
    df_x_fg = df_x.grid_fg
    df_x_pts = df_x.grid_pts
    # df_x_ppa = df_x.grid_ppa

    x_ssce = df_x_ssce.to_numpy()[0]
    x_prla = df_x_prla.to_numpy()[0]
    x_fga = df_x_fga.to_numpy()[0]
    x_fg = df_x_fg.to_numpy()[0]
    x_pts = df_x_pts.to_numpy()[0]
    # x_ppa = df_x_ppa.to_numpy()[0]

    for y in range(grid_y):
        for x in range(grid_x):
            locy = yy[y][x]
            locx = xx[y][x]
            dist = round(np.sqrt((locy**2)+(locx**2))/10,2)
            fga = x_fga[y][x]
            fg = x_fg[y][x]
            pts = x_pts[y][x]
            ssce = x_ssce[y][x]
            prla = x_prla[y][x]
    #         ppa = x_ppa[y][x]
            loc.append([locy + 2.5, locx + 2.5, fga, fg, pts, ssce, prla, dist])


    pd.DataFrame(loc, columns=['y', 'x', 'opp_fga', 'opp_fg', 'opp_pts', 'opp_ssce', 'opp_prla', 'dist']).to_csv('{}/to-map/teams/{}-opp-to-map.csv'.format(outdir, p), index=False)

In [10]:
filehandler_teams_opp = open('{}/teams-opp-local-stats.obj'.format(outdir), 'rb') 
df_stats_teams_opp = pickle.load(filehandler_teams_opp)

df_stats_teams_opp

,team,grid_ssce,grid_prla,grid_lppa,grid_elpts,grid_fg,grid_fga,grid_pts
0,NU,"[[-0.886, -0.887, -0.875, -0.816, -0.725, -0.6...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,UST,"[[-0.886, -0.887, -0.875, -0.816, -0.725, -0.6...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,FEU,"[[-0.886, -0.887, -0.875, -0.816, -0.725, -0.6...","[[-0.886, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.886, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,DLSU,"[[-0.886, -0.887, -0.875, -0.816, -0.725, -0.6...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,ADMU,"[[-0.886, -0.887, -0.875, -0.816, -0.725, -0.6...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
5,ADU,"[[-0.886, -0.887, -0.875, -0.816, -0.725, -0.6...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
6,UP,"[[-0.886, -0.887, -0.875, -0.816, -0.725, -0.6...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.591, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.591, 0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
7,UE,"[[-0.886, -0.887, -0.875, -0.816, -0.725, -0.6...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.591, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.591, 0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
